In [ ]:
%matplotlib inline

In [ ]:
from __future__ import (print_function, unicode_literals, division)

try:
    from urllib.parse import urlparse
except ImportError:
    from urlparse import urlparse
    
from itertools import islice
import sys

import numpy as np
import pandas as pd
from pandas import (DataFrame, Series)
import matplotlib.pyplot as plt

In [ ]:
# package up logic in a package
# from hypothesisapi import API

# include your hypothes.is USERNAME, TOKEN as parameters in a hypothesis_settings.py file in your sys.path
# get TOKEN at https://hypothes.is/profile/developer

from hypothesis_settings import (USERNAME, TOKEN)
from Hypothes_is import Hypothesis, HypothesisAnnotation

h_api = Hypothesis(USERNAME, TOKEN, max_results=sys.maxint)

# collect annotations for rdhyee

In [ ]:
# reading in all the annotations
annotations = list(h_api.search_all({'sort':'updated', 'order':'desc', 'user':'rdhyee@hypothes.is'}))

In [ ]:
len(annotations)

# collect all the public annotations

In [ ]:
from itertools import islice

annotations = []

for (i, annot) in enumerate(islice(h_api.search_all({'sort':'updated', 'order':'desc'}), 1000)):
    print("\r%d" % i , end="")
    annotations.append(annot)

In [ ]:
# write annotations out
import json
with open("annotations.json", "wb") as f:
    f.write(json.dumps(annotations))

In [ ]:
# read in annotations
import json
#annotations = json.loads(open("annotations.json").read())

In [ ]:
df = DataFrame(annotations)

In [ ]:
# date distribution
import dateutil.parser
s = df.created.apply(dateutil.parser.parse).apply(lambda d: (d.year, d.month)).value_counts()
s

In [ ]:
(s.sort_index(ascending=True).plot(kind='bar', color='green', # x='year/month', y='# of annotations'
                                  ))

In [ ]:
len(df.user.value_counts())

In [ ]:
df.user.value_counts()[:20]

In [ ]:
df.uri.apply(lambda url: urlparse(url)[1]).value_counts()

In [ ]:
# most annotated web pages
df.uri.value_counts()

# Writing annotations

In [ ]:
def test_create(h):
    return (h.create_annotation_with_target_using_only_text_quote(
       url="https://www.nytimes.com/2017/05/06/world/europe/france-election-emmanuel-macron-marine-le-pen.html",
       exact=u"""Then, barely an hour before the official close of campaigning at midnight Friday, the staff of the presumed front-runner, Emmanuel Macron, a 39-year-old former investment banker, announced that his campaign had been the target of a “massive and coordinated” hacking operation.""",
       prefix="for its raw anger and insolence.",
       suffix="Internal emails and other docume",
       text="hello",
       tags=()
    ))

In [ ]:
r = test_create(h_api)
r

In [ ]:
r.text